In [1]:
from protera_stability.proteins import EmbeddingExtractor1D

In [2]:
from pathlib import Path
import pandas as pd

data_path = Path("../data")

!ls $data_path

clustering	 stability.csv		       stability_test.h5
mmseq		 stability_diversity.csv       stability_train.csv
mmseq_q		 stability_embeddings.pkl      stability_train_diversity.csv
mmseq_t		 stability.h5		       stability_train.h5
raw		 stability_test.csv	       tmp
sequences.fasta  stability_test_diversity.csv


> * Los `.csv` deben contener las columnas `labels` y `sequences`.

> * `data_path` debe ser Path y un directorio que contenga tus `.csv`

In [3]:
pd.read_csv(data_path / "stability_train.csv")

,consensus_stability_score,sequence
0,-0.03,TELKKKLEEALKKGEEVRVKFNGIEIRNTSEDAARKAVELLEK
1,1.41,GSSQETIEVEDEEEARRVAKELRKTGYEVKIERRGNKWHVHRT
2,1.11,TTIHVGDLTLKYDNPKKAYEIAKKLAKKYNLQVTIKNGKITVT
3,1.24,SKDEAQREAERAIRSGNKEEARRILEEVGYSPEQAERIIRKLG
4,0.89,FEIPDDVPLPAGWEMARTSSGQRYFKNHIDQTTTWQDPRKAMLSQM
...,...,...
8199,1.59,GSSKTQYEYDTKEEAQKAYEKFKKQGIPVTITQKNGKWFVQVE
8200,0.78,TELKKALEEALKKGEEVRVKFNGIEIRITSEDAARKAVELLEK
8201,1.26,SKDEAQREAERAIRSGNKEEARRILEEAGYSPEQAERIARKLG
8202,1.48,GSSKTQYEYDTKEEAQPAYEKFKKQGIPVTITQKNGKWFVQVE


In [4]:
args_dict = {
    "model_name": "esm1b_t33_650M_UR50S",
    "base_path": data_path,
    "gpu": True,
}

emb_stabilty = EmbeddingExtractor1D(**args_dict)

Using cache found in /home/roberto/.cache/torch/hub/facebookresearch_esm_master


In [5]:
embeddings = emb_stabilty.generate_embeddings(
    ["stability_train.csv"],  # esta en data_path
    path_out="stability_embeddings_train",  # guardarlo en data_path/stability_train.pkl
    bs=256,
)

100%|██████████| 33/33 [00:56<00:00,  1.73s/it]

Embeddings saved to ../data/stability_embeddings_train.pkl


In [6]:
dset = emb_stabilty.generate_datasets(
    ["stability.csv"],
    h5_stem="stability",  # data_path / "stability".h5
    bs=256,
    embedding_file="stability_embeddings",  # cargar los embeddings desde donde se espera que esten
    target_name="stability_scores"
)

Returning existing dataset...


In [7]:
dset["labels"][:10], dset["sequences"][:10]

(array([ 0.37,  0.62, -0.03,  1.41,  1.11,  1.24,  1.05,  0.89,  0.88,
         1.15]),
 array([b'GSSQETIEVEDEEEARRVAKELRKKGYEVKDERRGNKWHVHRT',
        b'TLDEARELVERAKKEGTGMDVNGQRFEDWREAERWVREQEKNK',
        b'TELKKKLEEALKKGEEVRVKFNGIEIRNTSEDAARKAVELLEK',
        b'GSSQETIEVEDEEEARRVAKELRKTGYEVKIERRGNKWHVHRT',
        b'TTIHVGDLTLKYDNPKKAYEIAKKLAKKYNLQVTIKNGKITVT',
        b'SKDEAQREAERAIRSGNKEEARRILEEVGYSPEQAERIIRKLG',
        b'TIDEIIKALEQAVKDGKPIQVGNYTVTSADEAEKLAKKLKKEY',
        b'FEIPDDVPLPAGWEMARTSSGQRYFKNHIDQTTTWQDPRKAMLSQM',
        b'TTIHVGDLTLKYDNPKKAYEIAKKLDKKYNLTVTIKNGKITVT',
        b'GSSGSLSDEDFKAVFGMTRSAFAMLPLWKQQNLKKEKGLFGSS'], dtype=object))

In [8]:
dset.close() # hay que cerrarlo pq es un h5py